# pyWECcast - v0.2.2
## WEC Powerseries Generator

This version release represents a refactoring to reduce dependence on specific file configurations for powerseries generation.

This is also the first implementation of numba accelerated functions with parallel functionality included

This is an example on basic usage on the reconstruction flow and process


In [1]:
# General import directly from the local pyWECcast directory

import sys
sys.path.insert(1, '../')

import src as wc

In [2]:
baseDir = f'/mnt/c/Users/abharath/Documents/Projects/Grid_Value'
dataDir = f'{baseDir}/data'
buoyDB = f'{dataDir}/buoy_downloads.h5'

In [3]:
# First thing is first, we must compile the sea-state data we wish to use to base our WEC 
# power series on

# For a basic reconstruction we need waveheight and period timeseries, but for a directional
# reconstruction we will require directional data as well

# This example is based on the swell wave data contained within the NOAA Buoy data

from h5py import File

with File(buoyDB,'r') as buoys:
    buoy = buoys['46041']
    SwH, SwP, SwD = buoy['SwH'][:], buoy['SwP'][:], buoy['SwD'][:]
    time_index = buoy['time_index'][:]


In [4]:
# we also include convenience loading functions for NOAA buoy data files (based on those 
# constructed using the pyWECcast download utility)

Hs = 'SwH'
Tp = 'SwP'
Dir = 'SwD'

time_index, SwH, SwP, SwD = wc.extract_NOAA_buoy(buoyDB,'46041',Hs, Tp, Dir)


In [5]:
# for the particular NOAA buoy data files we have convenience functions in wc.buoy to convert to
# a numerical format.

timestamps = wc.ufuncs.binary_timestamp_convert(time_index)

sea_direction = wc.ufuncs.binary_direction_convert(SwD)


../src/ufuncs/utility_functions.py:77: UserWarning: Directional ndarray contains NaNs - default value inserted 0.0
  warnings.warn(warn,UserWarning)


In [6]:
# we can manipulate the data to reduce the amount of processing done in the reconstructions

cut = 5
timestamps, SwH, SwP = timestamps[:cut], SwH[:cut], SwP[:cut]

In [7]:
# With a WECSim powerseries matrix, we can link the sea-state data to the WEC simulations 
# that are available. The closest match can be determined using a nearest neighbor method on 
# separate Hs, Tp and direction values or combinations of the groups

wecSim = f'{dataDir}/WECSim_dataset_RM3_scale_0-1873.hdf5'

wecSimSeas, measured = wc.link_sea_states(wecSim,SwH,SwP)

# we can also compare WECSim sea states and those measure to analyze the variations between them

print([f'Hs - {abs(j[0][0]-j[1][0])} // Tp - {abs(j[0][1]-j[1][1])}'
            for i,j in enumerate(zip(wecSimSeas, measured))])

['Hs - 0.15000009536743164 // Tp - 0.10000038146972656', 'Hs - 0.25 // Tp - 0.3000001907348633', 'Hs - 0.04999995231628418 // Tp - 0.10000038146972656', 'Hs - 0.04999995231628418 // Tp - 0.3000001907348633', 'Hs - 0.15000009536743164 // Tp - 0.10000038146972656']


In [8]:
# With the sea-states in the dataset linked to what is available in the WECSim simulation db
# We can calculate the FFTs to obtian the reconstruction coefficients

fftFname = f'{dataDir}/testing_fft_db.h5'

# if inMemory:True, calculate_fft_matrix() will return coefficients as a dict
fft_matrix = wc.calculate_fft_matrix(wecSim,wecSimSeas[:,0],wecSimSeas[:,1],
                                  fftFname=fftFname,inMemory=True)

In [9]:
# if we are looking to simulate multiple WECs, there is a convinience function in place to
# setup the fft_matrix to do this.

nWECs = 25

fft_matrix = wc.multiple_realizations(nWECs,fft_matrix)


In [25]:
# With the coefficients and frequencies we can not reconstruct the powerseries
# This call will make nWECs reconstructions with a random phase applied to each

freq = '60S'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

random_phase, time = wc.construct_powerseries(timestamps,freq,wecSimSeas[:,0],wecSimSeas[:,1],
                         fft_matrix=fft_matrix,recFile=resultDB,inMemory=True,inPhase=False)


100%|██████████| 4/4 [00:12<00:00,  3.18s/it]


In [ ]:
# With the coefficients and frequencies we can not reconstruct the powerseries
# This call will make nWECs reconstructions with a common randomly chosen phase applied to each

freq = '60S'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

common_phase, time = wc.construct_powerseries(timestamps,freq,wecSimSeas[:,0],wecSimSeas[:,1],
                         fft_matrix=fft_matrix,recFile=resultDB,inMemory=True,inPhase=True)


In [16]:
# With the coefficients and frequencies we can not reconstruct the powerseries and set the seeds for the phases 
# used so that the data is reconstructable

from numpy import array, int64

freq = '60S'
resultDB = f'{dataDir}/testing_buoy_powerseries.h5'

# Note a seed needs to be passed as an int64 for each WEC but can be any value
Seeds = array([i for i in range(nWECs)],dtype=int64)

set_phase, time = wc.construct_powerseries(timestamps,freq,wecSimSeas[:,0],wecSimSeas[:,1],
                         fft_matrix=fft_matrix,recFile=resultDB,inMemory=True,Seeds=Seeds) 


100%|██████████| 4/4 [00:07<00:00,  1.99s/it]


In [17]:
%matplotlib widget

import numpy as np

import matplotlib.pyplot as plt

plotValues = set_phase

for i in range(results.shape[-1]):
    plt.plot(time,plotValues[:,i])
    
plt.plot(time,np.mean(plotValues,axis=1),color='k')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …